In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

In [ ]:

alldf = pd.read_parquet(os.path.join(hndl.sandbox_dir,'watershed_recs.parquet'))


In [ ]:
alldf.date = pd.to_datetime(alldf.date)
wsname = alldf.huc8_name.iloc[0]
wscode = alldf.huc8.iloc[0]
with open(os.path.join(hndl.sandbox_dir,'watershed_state.txt'),'r') as f:
          statename = f.read()
statename = statename.strip()
# statename = alldf.bgStateName.iloc[0]
# countyname = alldf.bgCountyName.iloc[0]
#print(statename, countyname, len(alldf))
# print(alldf.columns)

In [ ]:
name = f'Watershed: {wsname.title()} - {wscode},<br>{statename.title()}'
showHeader(name, link_up_level=1)

# Where are the fracking locations in this watershed?
This is not an exhaustive set of wells in this watershed; it is only those wells for which an operating company submits a chemical disclosure to FracFocus.  In addition, this map omits disclosures for which location information is conflicting, such as the Latitude/Longitude values are outside of the reported county.

In [ ]:
import geopandas as gpd
final_crs = 4326 # WGS84

fn = r"C:\MyDocs\OpenFF\data\non-FF\georef-united-states-of-america-state.geojson"
geojson = gpd.read_file(fn)
geojson['StateName'] = geojson.ste_name.str.lower()
# geojson['CountyName'] = geojson.coty_name.str.lower()
# geojson = mapping.fix_county_names(geojson)
# geojson = geojson[(geojson.StateName==statename)&(geojson.CountyName==countyname)]
geojson = geojson[geojson.StateName==statename]
shape_flag = len(geojson)==1
# print(shape_flag, len(geojson))

# #!!!! Need to move this to the caller! don't run with every watershed!
# gdb_path =  r"C:\MyDocs\integrated\ext_data\WBD_National_GDB.zip"
# huc8_layer_name = 'WBDHU8'  # Example layer name, use the one you found
# #!!!!!

# # Read the specific layer into a GeoDataFrame
# huc8_gdf = gpd.read_file(gdb_path, layer=huc8_layer_name)
# huc8_gdf.to_crs(final_crs)

huc8_gdf = gpd.read_parquet(os.path.join(hndl.sandbox_dir,'huc8_gdf.parquet'))
huc8_gdf['huc8_name'] = huc8_gdf['name']

watershed_gdf = huc8_gdf[huc8_gdf.huc8==wscode][['geometry']]
include_filled_shape = True
# print(len(watershed_gdf))

In [ ]:
# gb = alldf[['DisclosureId','bgLatitude','bgLongitude','APINumber','TBWV','OperatorName','WellName','year']]\
#              .groupby('DisclosureId',as_index=False)[['bgLatitude','bgLongitude','APINumber','TBWV','OperatorName','WellName','year']].first()

# get the disclosure level data set
gb = pd.read_parquet(os.path.join(hndl.sandbox_dir,'watershed.parquet'))

mapping.create_integrated_point_map(gb,
                 aliases=['API Number:','Well Name:','Water used:','Operator:','date:'],
                 fields= ['APINumber','WellName','TBWV','OperatorName','year'],
                 include_shape = shape_flag,
                 area_df = geojson,
                 include_filled_shape = include_filled_shape,
                 filled_area_df = watershed_gdf
                )

## Summary of all reported chemicals in this watershed


In [ ]:
import openFF.common.chem_list_summary as cls

try:
    chemsum = cls.ChemListSummary(df=alldf)
    out = chemsum.get_display_table(colset='colab_v1')
    iShow(out.drop(['refs','rq_lbs','extrnl'],axis=1).reset_index(drop=True))
except:
    display(md('## No Data to Summarize !'))

## List of wells in this watershed

In [ ]:
def make_disc_link(row):
    return th.getDisclosureLink(row.APINumber,row.DisclosureId,'Open-FF link')
alldf['disc_link'] = alldf.apply(lambda x: make_disc_link(x),axis=1)
def make_FF_link(row):
    return th.getFFLink(row, fmt='short')
alldf['API Number'] = alldf.apply(lambda x: make_FF_link(x),axis=1)
alldf['year'] = alldf.date.dt.year

In [ ]:
iShow(alldf[['DisclosureId','API Number','disc_link','bgOperatorName','year']]\
                  .groupby('DisclosureId',as_index=False)[['API Number','disc_link','bgOperatorName','year']].first())